In [1]:
import transformers
print(transformers.__version__)


c:\Users\akqp4\python_projects\NLP\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.57.1


In [2]:
import re
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm


file_path = r"Preprocessed_QA_Dataset.csv"
output_dir = r"NLP"
model_name = "allenai/scibert_scivocab_uncased"
batch_size = 32 

# Device and model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.to(device)
model.eval()

# Utilities

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # (batch_size, seq_len, hidden)
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * mask, dim=1)
    sum_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask

def batch_embed_texts(texts, batch_size=batch_size):
    """Return numpy array of shape (len(texts), hidden_size)."""
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Batches", leave=False):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True,
                           padding=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        emb = mean_pooling(outputs, inputs['attention_mask'])
        emb = torch.nn.functional.normalize(emb, p=2, dim=1)
        all_embs.append(emb.cpu().numpy())
    if len(all_embs) == 0:
        return np.zeros((0, model.config.hidden_size), dtype=np.float32)
    return np.vstack(all_embs)


# Load dataset & detect columns

df = pd.read_csv(file_path)
# normalize column names
orig_columns = df.columns.tolist()
df.columns = df.columns.str.strip().str.lower()

print("Detected columns:", df.columns.tolist())

q_pattern = re.compile(r"^question(\d+)$")
a_pattern = re.compile(r"^answer(\d+)$")

q_cols = {}
a_cols = {}
for col in df.columns:
    m = q_pattern.match(col)
    if m:
        q_cols[int(m.group(1))] = col
    m2 = a_pattern.match(col)
    if m2:
        a_cols[int(m2.group(1))] = col

paired_indices = sorted(set(q_cols.keys()) & set(a_cols.keys()))
if not paired_indices:
    raise ValueError("No matching questionX/answerX column pairs found in the CSV.")

print("Found paired indices:", paired_indices)


# Process each pair

os.makedirs(output_dir, exist_ok=True)
summary = []

for idx in paired_indices:
    q_col = q_cols[idx]
    a_col = a_cols[idx]
    print(f"\nProcessing pair: {q_col} <-> {a_col}")

    q_texts = df[q_col].fillna("").astype(str).tolist()
    a_texts = df[a_col].fillna("").astype(str).tolist()

    # Embed questions
    print(f" - Embedding {len(q_texts)} texts from '{q_col}'")
    q_embs = batch_embed_texts(q_texts, batch_size=batch_size)

    # Embed answers
    print(f" - Embedding {len(a_texts)} texts from '{a_col}'")
    a_embs = batch_embed_texts(a_texts, batch_size=batch_size)

    # Save per-pair files
    q_npy = os.path.join(output_dir, f"{q_col}_embeddings.npy")
    a_npy = os.path.join(output_dir, f"{a_col}_embeddings.npy")
    np.save(q_npy, q_embs)
    np.save(a_npy, a_embs)

    pd.DataFrame(q_embs).to_csv(os.path.join(output_dir, f"{q_col}_embeddings.csv"), index=False)
    pd.DataFrame(a_embs).to_csv(os.path.join(output_dir, f"{a_col}_embeddings.csv"), index=False)

    summary.append((idx, q_col, a_col, q_embs.shape, a_embs.shape))
    print(f" -> Saved: {q_npy} ({q_embs.shape}), {a_npy} ({a_embs.shape})")


print("\nProcessing complete. Summary:")
for s in summary:
    print(f"Pair {s[0]}: {s[1]} <-> {s[2]} | shapes: {s[3]} , {s[4]}")

print("\nAll embeddings saved to:", output_dir)


Using device: cuda
Detected columns: ['question1', 'question2', 'question3', 'answer1', 'answer2', 'answer3']
Found paired indices: [1, 2, 3]

Processing pair: question1 <-> answer1
 - Embedding 4118 texts from 'question1'


 - Embedding 4118 texts from 'answer1'


 -> Saved: NLP\question1_embeddings.npy ((4118, 768)), NLP\answer1_embeddings.npy ((4118, 768))

Processing pair: question2 <-> answer2
 - Embedding 4118 texts from 'question2'


 - Embedding 4118 texts from 'answer2'


 -> Saved: NLP\question2_embeddings.npy ((4118, 768)), NLP\answer2_embeddings.npy ((4118, 768))

Processing pair: question3 <-> answer3
 - Embedding 4118 texts from 'question3'


 - Embedding 4118 texts from 'answer3'


 -> Saved: NLP\question3_embeddings.npy ((4118, 768)), NLP\answer3_embeddings.npy ((4118, 768))

Processing complete. Summary:
Pair 1: question1 <-> answer1 | shapes: (4118, 768) , (4118, 768)
Pair 2: question2 <-> answer2 | shapes: (4118, 768) , (4118, 768)
Pair 3: question3 <-> answer3 | shapes: (4118, 768) , (4118, 768)

All embeddings saved to: NLP


In [6]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import chromadb
from chromadb.config import Settings
from tqdm import tqdm

# ==================== CONFIGURATION ====================
csv_file = "preprocessed_science_qa.csv"  # Use preprocessed dataset
vector_db_dir = "vector_db"
model_name = "allenai/scibert_scivocab_uncased"
batch_size = 32

# Create vector DB directory
os.makedirs(vector_db_dir, exist_ok=True)

# ==================== INITIALIZE SCIBERT ====================
print("="*70)
print("SCIBERT EMBEDDING GENERATION & CHROMADB STORAGE")
print("="*70)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

print("\nLoading SciBERT model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.to(device)
model.eval()
print("✓ Model loaded successfully")

# ==================== EMBEDDING UTILITIES ====================
def mean_pooling(model_output, attention_mask):
    """Mean pooling over token embeddings."""
    token_embeddings = model_output.last_hidden_state
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * mask, dim=1)
    sum_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask

def batch_embed_texts(texts, batch_size=32):
    """
    Embed a list of texts using SciBERT in batches.
    Returns: numpy array of shape (len(texts), 768)
    """
    all_embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding", leave=False):
        batch_texts = texts[i:i+batch_size]
        
        # Tokenize
        inputs = tokenizer(
            batch_texts, 
            return_tensors="pt", 
            truncation=True,
            padding=True, 
            max_length=512
        ).to(device)
        
        # Generate embeddings
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Mean pooling
        embeddings = mean_pooling(outputs, inputs['attention_mask'])
        
        # Normalize
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        
        all_embeddings.append(embeddings.cpu().numpy())
    
    if len(all_embeddings) == 0:
        return np.zeros((0, model.config.hidden_size), dtype=np.float32)
    
    return np.vstack(all_embeddings)

# ==================== INITIALIZE CHROMADB ====================
print("\nInitializing ChromaDB...")

try:
    settings = Settings(
        anonymized_telemetry=False,
        allow_reset=True,
        is_persistent=True
    )
    client = chromadb.PersistentClient(path=vector_db_dir, settings=settings)
    print(f"✓ Connected to ChromaDB at: {vector_db_dir}")
except Exception as e:
    print(f"⚠ Error connecting to ChromaDB: {e}")
    print("Attempting to fix lock issues...")
    
    # Try to remove lock files
    import glob
    lock_files = glob.glob(os.path.join(vector_db_dir, "*.lock"))
    for lock_file in lock_files:
        try:
            os.remove(lock_file)
            print(f"  Removed: {lock_file}")
        except:
            pass
    
    # Try again
    client = chromadb.PersistentClient(path=vector_db_dir, settings=settings)
    print("✓ Connected successfully after cleanup")

# Create collections
print("\nCreating/loading collections...")

# Collection for answers (with embeddings)
answers_collection = client.get_or_create_collection(
    name="science_qa_answers",
    metadata={"description": "Answer embeddings from science Q&A dataset"}
)
print(f"✓ Answers collection: {answers_collection.count()} existing documents")

# Collection for questions (for topic clustering and retrieval)
questions_collection = client.get_or_create_collection(
    name="science_qa_questions",
    metadata={"description": "Question embeddings for topic clustering"}
)
print(f"✓ Questions collection: {questions_collection.count()} existing documents")

# ==================== LOAD DATASET ====================
print("\n" + "="*70)
print("LOADING PREPROCESSED DATASET")
print("="*70)

df = pd.read_csv(csv_file)
print(f"\nDataset loaded: {len(df)} rows")
print(f"Columns: {df.columns.tolist()}")

# Detect question and answer columns (case-insensitive)
question_col = None
answer_col = None

for col in df.columns:
    if col.lower() == 'question':
        question_col = col
    elif col.lower() == 'answer':
        answer_col = col

if not question_col or not answer_col:
    raise ValueError(f"Could not find 'question' and 'answer' columns! Found: {df.columns.tolist()}")

print(f"\nUsing columns:")
print(f"  Questions: '{question_col}'")
print(f"  Answers: '{answer_col}'")

# Get data
questions = df[question_col].fillna("").astype(str).tolist()
answers = df[answer_col].fillna("").astype(str).tolist()

# Filter out empty entries
valid_indices = [
    i for i in range(len(answers)) 
    if answers[i].strip() and questions[i].strip()
]

print(f"\nTotal rows: {len(questions)}")
print(f"Valid entries (non-empty Q&A): {len(valid_indices)}")

if not valid_indices:
    raise ValueError("No valid question-answer pairs found!")

# ==================== EMBED QUESTIONS ====================
print("\n" + "="*70)
print("EMBEDDING QUESTIONS")
print("="*70)

valid_questions = [questions[i] for i in valid_indices]
print(f"\nEmbedding {len(valid_questions)} questions...")
question_embeddings = batch_embed_texts(valid_questions, batch_size=batch_size)
print(f"✓ Generated embeddings: {question_embeddings.shape}")

# ==================== EMBED ANSWERS ====================
print("\n" + "="*70)
print("EMBEDDING ANSWERS")
print("="*70)

valid_answers = [answers[i] for i in valid_indices]
print(f"\nEmbedding {len(valid_answers)} answers...")
answer_embeddings = batch_embed_texts(valid_answers, batch_size=batch_size)
print(f"✓ Generated embeddings: {answer_embeddings.shape}")

# ==================== STORE ANSWERS IN CHROMADB ====================
print("\n" + "="*70)
print("STORING ANSWERS IN CHROMADB")
print("="*70)

answer_ids = []
answer_documents = []
answer_metadatas = []
answer_embeddings_list = []

for i, orig_idx in enumerate(valid_indices):
    doc_id = f"answer_{orig_idx}"
    
    # Check if already exists
    try:
        existing = answers_collection.get(ids=[doc_id])
        if existing['ids']:
            continue  # Skip if already exists
    except:
        pass
    
    answer_ids.append(doc_id)
    answer_documents.append(valid_answers[i])
    answer_metadatas.append({
        "row_index": int(orig_idx),
        "question": valid_questions[i],
        "answer_length": len(valid_answers[i].split())
    })
    answer_embeddings_list.append(answer_embeddings[i].tolist())

# Add to ChromaDB in batches
if answer_ids:
    batch_size_db = 1000
    print(f"\nAdding {len(answer_ids)} new answer documents...")
    
    for i in tqdm(range(0, len(answer_ids), batch_size_db), desc="Answer Batches"):
        batch_ids = answer_ids[i:i+batch_size_db]
        batch_docs = answer_documents[i:i+batch_size_db]
        batch_meta = answer_metadatas[i:i+batch_size_db]
        batch_embs = answer_embeddings_list[i:i+batch_size_db]
        
        try:
            answers_collection.add(
                ids=batch_ids,
                documents=batch_docs,
                metadatas=batch_meta,
                embeddings=batch_embs
            )
        except Exception as e:
            print(f"\n⚠ Error adding batch: {e}")
    
    print(f"✓ Stored {len(answer_ids)} answers")
else:
    print("\nℹ All answers already exist in database")

# ==================== STORE QUESTIONS IN CHROMADB ====================
print("\n" + "="*70)
print("STORING QUESTIONS IN CHROMADB")
print("="*70)

question_ids = []
question_documents = []
question_metadatas = []
question_embeddings_list = []

for i, orig_idx in enumerate(valid_indices):
    doc_id = f"question_{orig_idx}"
    
    # Check if already exists
    try:
        existing = questions_collection.get(ids=[doc_id])
        if existing['ids']:
            continue
    except:
        pass
    
    question_ids.append(doc_id)
    question_documents.append(valid_questions[i])
    question_metadatas.append({
        "row_index": int(orig_idx),
        "answer": valid_answers[i][:200],  # Store snippet of answer
        "question_length": len(valid_questions[i].split())
    })
    question_embeddings_list.append(question_embeddings[i].tolist())

# Add to ChromaDB in batches
if question_ids:
    print(f"\nAdding {len(question_ids)} new question documents...")
    
    for i in tqdm(range(0, len(question_ids), batch_size_db), desc="Question Batches"):
        batch_ids = question_ids[i:i+batch_size_db]
        batch_docs = question_documents[i:i+batch_size_db]
        batch_meta = question_metadatas[i:i+batch_size_db]
        batch_embs = question_embeddings_list[i:i+batch_size_db]
        
        try:
            questions_collection.add(
                ids=batch_ids,
                documents=batch_docs,
                metadatas=batch_meta,
                embeddings=batch_embs
            )
        except Exception as e:
            print(f"\n⚠ Error adding batch: {e}")
    
    print(f"✓ Stored {len(question_ids)} questions")
else:
    print("\nℹ All questions already exist in database")

# ==================== SUMMARY ====================
print("\n" + "="*70)
print("SUMMARY")
print("="*70)

print(f"\n📊 Processing Complete:")
print(f"   Valid Q&A pairs processed: {len(valid_indices)}")
print(f"   New answers stored: {len(answer_ids) if answer_ids else 0}")
print(f"   New questions stored: {len(question_ids) if question_ids else 0}")

print(f"\n📊 ChromaDB Collections:")
print(f"   Answers collection: {answers_collection.count()} total documents")
print(f"   Questions collection: {questions_collection.count()} total documents")

print(f"\n💾 Database location: {vector_db_dir}")

# ==================== TEST QUERY ====================
print("\n" + "="*70)
print("TESTING DATABASE")
print("="*70)

if answers_collection.count() > 0 and questions_collection.count() > 0:
    print("\nPerforming test query...")
    
    # Get first question
    first_question_data = questions_collection.get(
        limit=1,
        include=['embeddings', 'documents', 'metadatas']
    )
    
    if first_question_data['embeddings']:
        test_question = first_question_data['documents'][0]
        test_embedding = first_question_data['embeddings'][0]
        
        print(f"\nTest question: {test_question[:100]}...")
        
        # Query for similar answers
        results = answers_collection.query(
            query_embeddings=[test_embedding],
            n_results=3,
            include=['documents', 'metadatas', 'distances']
        )
        
        print("\nTop 3 most similar answers:")
        for i, (doc, metadata, distance) in enumerate(zip(
            results['documents'][0],
            results['metadatas'][0],
            results['distances'][0]
        )):
            similarity = 1 - (distance ** 2) / 2
            print(f"\n{i+1}. Similarity: {similarity:.2%}")
            print(f"   Answer: {doc[:100]}...")
            print(f"   From Row: {metadata['row_index']}")
        
        print("\n✓ Database test successful!")
    else:
        print("⚠ No embeddings found for testing")
else:
    print("⚠ Database is empty, cannot perform test query")

# ==================== USAGE INSTRUCTIONS ====================
print("\n" + "="*70)
print("USAGE INSTRUCTIONS")
print("="*70)

print("""
The database is now ready! You can use it with:

1. Load the collections:
   import chromadb
   client = chromadb.PersistentClient(path="vector_db")
   answers_collection = client.get_collection("science_qa_answers")
   questions_collection = client.get_collection("science_qa_questions")

2. Query for similar answers (given a question embedding):
   results = answers_collection.query(
       query_embeddings=[your_question_embedding],
       n_results=5
   )

3. Find similar questions (for topic clustering):
   similar_questions = questions_collection.query(
       query_embeddings=[current_question_embedding],
       n_results=10
   )

4. Get specific document by ID:
   doc = answers_collection.get(ids=["answer_42"])

5. Search by text (ChromaDB will embed it for you):
   results = answers_collection.query(
       query_texts=["What is photosynthesis?"],
       n_results=5
   )
""")

print("\n" + "="*70)
print("✓✓✓ COMPLETE ✓✓✓")
print("="*70)

SCIBERT EMBEDDING GENERATION & CHROMADB STORAGE

Using device: cuda

Loading SciBERT model...
✓ Model loaded successfully

Initializing ChromaDB...
✓ Connected to ChromaDB at: vector_db

Creating/loading collections...
✓ Answers collection: 0 existing documents
✓ Questions collection: 0 existing documents

LOADING PREPROCESSED DATASET

Dataset loaded: 808 rows
Columns: ['Question', 'Answer']

Using columns:
  Questions: 'Question'
  Answers: 'Answer'

Total rows: 808
Valid entries (non-empty Q&A): 808

EMBEDDING QUESTIONS

Embedding 808 questions...


✓ Generated embeddings: (808, 768)

EMBEDDING ANSWERS

Embedding 808 answers...


✓ Generated embeddings: (808, 768)

STORING ANSWERS IN CHROMADB

Adding 808 new answer documents...


Answer Batches: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]


✓ Stored 808 answers

STORING QUESTIONS IN CHROMADB

Adding 808 new question documents...


Question Batches: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s]

✓ Stored 808 questions

SUMMARY

📊 Processing Complete:
   Valid Q&A pairs processed: 808
   New answers stored: 808
   New questions stored: 808

📊 ChromaDB Collections:
   Answers collection: 808 total documents
   Questions collection: 808 total documents

💾 Database location: vector_db

TESTING DATABASE

Performing test query...


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import chromadb
from chromadb.config import Settings
from tqdm import tqdm

# ==================== CONFIGURATION ====================
csv_file = "preprocessed_science_qa.csv"  # Use preprocessed dataset
vector_db_dir = "vector_db"
model_name = "allenai/scibert_scivocab_uncased"
batch_size = 32

# Create vector DB directory
os.makedirs(vector_db_dir, exist_ok=True)

# ==================== INITIALIZE SCIBERT ====================
print("="*70)
print("SCIBERT EMBEDDING GENERATION & CHROMADB STORAGE (FIXED)")
print("="*70)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

print("\nLoading SciBERT model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.to(device)
model.eval()
print("[SUCCESS] Model loaded successfully")

# ==================== EMBEDDING UTILITIES ====================
def mean_pooling(model_output, attention_mask):
    """Mean pooling over token embeddings."""
    token_embeddings = model_output.last_hidden_state
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * mask, dim=1)
    sum_mask = torch.clamp(mask.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask

def batch_embed_texts(texts, batch_size=32):
    """
    Embed a list of texts using SciBERT in batches.
    Returns: numpy array of shape (len(texts), 768)
    """
    all_embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding", leave=False):
        batch_texts = texts[i:i+batch_size]
        
        # Tokenize
        inputs = tokenizer(
            batch_texts, 
            return_tensors="pt", 
            truncation=True,
            padding=True, 
            max_length=512
        ).to(device)
        
        # Generate embeddings
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Mean pooling
        embeddings = mean_pooling(outputs, inputs['attention_mask'])
        
        # Normalize
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        
        all_embeddings.append(embeddings.cpu().numpy())
    
    if len(all_embeddings) == 0:
        return np.zeros((0, model.config.hidden_size), dtype=np.float32)
    
    return np.vstack(all_embeddings)

# ==================== INITIALIZE CHROMADB ====================
print("\nInitializing ChromaDB...")

try:
    settings = Settings(
        anonymized_telemetry=False,
        allow_reset=True,
        is_persistent=True
    )
    client = chromadb.PersistentClient(path=vector_db_dir, settings=settings)
    print(f"[SUCCESS] Connected to ChromaDB at: {vector_db_dir}")
except Exception as e:
    print(f"[WARNING] Error connecting to ChromaDB: {e}")
    print("Attempting to fix lock issues...")
    
    # Try to remove lock files
    import glob
    lock_files = glob.glob(os.path.join(vector_db_dir, "*.lock"))
    for lock_file in lock_files:
        try:
            os.remove(lock_file)
            print(f"  Removed: {lock_file}")
        except:
            pass
    
    # Try again
    client = chromadb.PersistentClient(path=vector_db_dir, settings=settings)
    print("[SUCCESS] Connected successfully after cleanup")

# Delete old collections and create new ones
print("\n[DELETE] Deleting old collections...")
try:
    client.delete_collection("science_qa_answers")
    print("  Deleted: science_qa_answers")
except:
    pass

try:
    client.delete_collection("science_qa_questions")
    print("  Deleted: science_qa_questions")
except:
    pass

print("\nCreating new collections...")

# Collection for answers (indexed by QUESTION embeddings)
answers_collection = client.create_collection(
    name="science_qa_answers",
    metadata={"description": "Answers indexed by question embeddings (FIXED)"}
)
print(f"[SUCCESS] Answers collection created")

# Collection for questions (for topic clustering and retrieval)
questions_collection = client.create_collection(
    name="science_qa_questions",
    metadata={"description": "Question embeddings for topic clustering"}
)
print(f"[SUCCESS] Questions collection created")

# ==================== LOAD DATASET ====================
print("\n" + "="*70)
print("LOADING PREPROCESSED DATASET")
print("="*70)

df = pd.read_csv(csv_file)
print(f"\nDataset loaded: {len(df)} rows")
print(f"Columns: {df.columns.tolist()}")

# Detect question and answer columns (case-insensitive)
question_col = None
answer_col = None

for col in df.columns:
    if col.lower() == 'question':
        question_col = col
    elif col.lower() == 'answer':
        answer_col = col

if not question_col or not answer_col:
    raise ValueError(f"Could not find 'question' and 'answer' columns! Found: {df.columns.tolist()}")

print(f"\nUsing columns:")
print(f"  Questions: '{question_col}'")
print(f"  Answers: '{answer_col}'")

# Get data and keep original order
questions = df[question_col].fillna("").astype(str).tolist()
answers = df[answer_col].fillna("").astype(str).tolist()

# Filter out empty entries but PRESERVE ORDER
valid_pairs = []
for i in range(len(questions)):
    q = questions[i].strip()
    a = answers[i].strip()
    if q and a and q.lower() != 'nan' and a.lower() != 'nan':
        valid_pairs.append({
            'row_index': i,
            'question': q,
            'answer': a
        })

print(f"\nTotal rows: {len(questions)}")
print(f"Valid entries (non-empty Q&A): {len(valid_pairs)}")

if not valid_pairs:
    raise ValueError("No valid question-answer pairs found!")

# ==================== EMBED QUESTIONS ====================
print("\n" + "="*70)
print("EMBEDDING QUESTIONS")
print("="*70)

valid_questions = [pair['question'] for pair in valid_pairs]
print(f"\nEmbedding {len(valid_questions)} questions...")
question_embeddings = batch_embed_texts(valid_questions, batch_size=batch_size)
print(f"[SUCCESS] Generated embeddings: {question_embeddings.shape}")

# ==================== EMBED ANSWERS ====================
print("\n" + "="*70)
print("EMBEDDING ANSWERS")
print("="*70)

valid_answers = [pair['answer'] for pair in valid_pairs]
print(f"\nEmbedding {len(valid_answers)} answers...")
answer_embeddings = batch_embed_texts(valid_answers, batch_size=batch_size)
print(f"[SUCCESS] Generated embeddings: {answer_embeddings.shape}")

# ==================== STORE IN CHROMADB (FIXED WAY) ====================
print("\n" + "="*70)
print("STORING Q&A PAIRS IN CHROMADB (FIXED)")
print("="*70)

print("\n[FIX] KEY FIX: Answers will be indexed by QUESTION embeddings!")
print("   This ensures when you query with a question, you get the paired answer.\n")

answer_ids = []
answer_documents = []
answer_metadatas = []
answer_embeddings_list = []

question_ids = []
question_documents = []
question_metadatas = []
question_embeddings_list = []

for i, pair in enumerate(valid_pairs):
    row_idx = pair['row_index']
    
    # For ANSWERS: Store answer TEXT with QUESTION embedding
    answer_ids.append(f"answer_{row_idx}")
    answer_documents.append(pair['answer'])
    answer_metadatas.append({
        "row_index": int(row_idx),
        "question": pair['question'][:200],  # Store question snippet for reference
        "answer_length": len(pair['answer'].split())
    })
    # CRITICAL: Use QUESTION embedding to index the ANSWER
    answer_embeddings_list.append(question_embeddings[i].tolist())
    
    # For QUESTIONS: Store question TEXT with QUESTION embedding
    question_ids.append(f"question_{row_idx}")
    question_documents.append(pair['question'])
    question_metadatas.append({
        "row_index": int(row_idx),
        "answer_snippet": pair['answer'][:200],
        "question_length": len(pair['question'].split())
    })
    question_embeddings_list.append(question_embeddings[i].tolist())

# Add to ChromaDB in batches
batch_size_db = 1000

print(f"\nAdding {len(answer_ids)} answers (indexed by question embeddings)...")
for i in tqdm(range(0, len(answer_ids), batch_size_db), desc="Answer Batches"):
    batch_ids = answer_ids[i:i+batch_size_db]
    batch_docs = answer_documents[i:i+batch_size_db]
    batch_meta = answer_metadatas[i:i+batch_size_db]
    batch_embs = answer_embeddings_list[i:i+batch_size_db]
    
    try:
        answers_collection.add(
            ids=batch_ids,
            documents=batch_docs,
            metadatas=batch_meta,
            embeddings=batch_embs
        )
    except Exception as e:
        print(f"\n[WARNING] Error adding batch: {e}")

print(f"[SUCCESS] Stored {len(answer_ids)} answers")

print(f"\nAdding {len(question_ids)} questions...")
for i in tqdm(range(0, len(question_ids), batch_size_db), desc="Question Batches"):
    batch_ids = question_ids[i:i+batch_size_db]
    batch_docs = question_documents[i:i+batch_size_db]
    batch_meta = question_metadatas[i:i+batch_size_db]
    batch_embs = question_embeddings_list[i:i+batch_size_db]
    
    try:
        questions_collection.add(
            ids=batch_ids,
            documents=batch_docs,
            metadatas=batch_meta,
            embeddings=batch_embs
        )
    except Exception as e:
        print(f"\n[WARNING] Error adding batch: {e}")

print(f"[SUCCESS] Stored {len(question_ids)} questions")

# ==================== VERIFICATION ====================
print("\n" + "="*70)
print("VERIFICATION")
print("="*70)

print("\nTesting Q&A matching with first 3 pairs...")

for test_idx in range(min(3, len(valid_pairs))):
    pair = valid_pairs[test_idx]
    row_idx = pair['row_index']
    
    print(f"\n{'─'*70}")
    print(f"Test {test_idx + 1}: Row {row_idx}")
    print(f"{'─'*70}")
    print(f"Question: {pair['question'][:80]}...")
    print(f"Answer: {pair['answer'][:80]}...")
    
    # Query with question embedding
    test_q_embedding = question_embeddings[test_idx]
    results = answers_collection.query(
        query_embeddings=[test_q_embedding.tolist()],
        n_results=1,
        include=['documents', 'metadatas', 'distances']
    )
    
    if results['ids'][0]:
        matched_row = results['metadatas'][0][0].get('row_index', -1)
        distance = results['distances'][0][0]
        similarity = 1 - (distance ** 2) / 2
        
        print(f"\nMatched row_index: {matched_row}")
        print(f"Similarity: {similarity:.4f}")
        
        if matched_row == row_idx:
            print("[PASS] CORRECT! Matched to same row")
        else:
            print(f"[FAIL] WRONG! Expected row {row_idx}, got {matched_row}")
    else:
        print("[FAIL] No results found!")

# ==================== SUMMARY ====================
print("\n" + "="*70)
print("SUMMARY")
print("="*70)

print(f"\n[STATS] Processing Complete:")
print(f"   Valid Q&A pairs processed: {len(valid_pairs)}")
print(f"   Answers stored: {len(answer_ids)}")
print(f"   Questions stored: {len(question_ids)}")

print(f"\n[STATS] ChromaDB Collections:")
print(f"   Answers collection: {answers_collection.count()} documents")
print(f"   Questions collection: {questions_collection.count()} documents")

print(f"\n[STORAGE] Database location: {vector_db_dir}")

print("\n" + "="*70)
print("[FIX] KEY FIX APPLIED:")
print("="*70)
print("""
BEFORE (Wrong):
  - Answers indexed by answer embeddings
  - Query with question -> finds semantically similar answer (WRONG pair!)

AFTER (Fixed):
  - Answers indexed by QUESTION embeddings
  - Query with question -> finds the PAIRED answer (CORRECT!)

This ensures Q&A pairs stay matched correctly!
""")

print("\n" + "="*70)
print("USAGE INSTRUCTIONS")
print("="*70)

print("""
Now when you query:

1. Get question embedding:
   q_embedding = get_embedding(question_text)

2. Query answers collection:
   results = answers_collection.query(
       query_embeddings=[q_embedding],
       n_results=1
   )
   
3. You'll get the PAIRED answer for that question!

Note: You can also use the row_index fallback method in your quiz
for guaranteed correct matching.
""")

print("\n" + "="*70)
print("[COMPLETE] PROCESS FINISHED")
print("="*70)

c:\Users\akqp4\python_projects\NLP\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SCIBERT EMBEDDING GENERATION & CHROMADB STORAGE (FIXED)

Using device: cuda

Loading SciBERT model...
✓ Model loaded successfully

Initializing ChromaDB...
✓ Connected to ChromaDB at: vector_db

🗑️  Deleting old collections...
  Deleted: science_qa_answers
  Deleted: science_qa_questions

Creating new collections...
✓ Answers collection created
✓ Questions collection created

LOADING PREPROCESSED DATASET

Dataset loaded: 980 rows
Columns: ['Question', 'Answer']

Using columns:
  Questions: 'Question'
  Answers: 'Answer'

Total rows: 980
Valid entries (non-empty Q&A): 980

EMBEDDING QUESTIONS

Embedding 980 questions...


✓ Generated embeddings: (980, 768)

EMBEDDING ANSWERS

Embedding 980 answers...


✓ Generated embeddings: (980, 768)

STORING Q&A PAIRS IN CHROMADB (FIXED)

🔧 KEY FIX: Answers will be indexed by QUESTION embeddings!
   This ensures when you query with a question, you get the paired answer.


Adding 980 answers (indexed by question embeddings)...


Answer Batches: 100%|██████████| 1/1 [00:00<00:00,  2.43it/s]


✓ Stored 980 answers

Adding 980 questions...


Question Batches: 100%|██████████| 1/1 [00:00<00:00,  2.48it/s]

✓ Stored 980 questions

VERIFICATION

Testing Q&A matching with first 3 pairs...

──────────────────────────────────────────────────────────────────────
Test 1: Row 0
──────────────────────────────────────────────────────────────────────
Question: heisenberg uncertainty principle...
Answer: state position momentum particle precisely know simultaneously accurately one kn...

Matched row_index: 0
Similarity: 1.0000
✅ CORRECT! Matched to same row

──────────────────────────────────────────────────────────────────────
Test 2: Row 1
──────────────────────────────────────────────────────────────────────
Question: chemical formula rustfeo iron oxide primary function mitochondrion cell...
Answer: generate energy atp cellular respiration often call powerhouse cell...

Matched row_index: 1
Similarity: 1.0000
✅ CORRECT! Matched to same row

──────────────────────────────────────────────────────────────────────
Test 3: Row 2
──────────────────────────────────────────────────────────────────────
Qu

In [13]:
import chromadb
import numpy as np

# ==================== CONFIGURATION ====================
vector_db_dir = "vector_db"

# ==================== CONNECT TO CHROMADB ====================
print("="*70)
print("TESTING CHROMADB DATABASE")
print("="*70)

print("\nConnecting to ChromaDB...")
print(f"✓ Connected to: {vector_db_dir}")

# Load collections
answers_collection = client.get_collection("science_qa_answers")
questions_collection = client.get_collection("science_qa_questions")

print(f"\n📊 Collection Status:")
print(f"   Answers: {answers_collection.count()} documents")
print(f"   Questions: {questions_collection.count()} documents")

# ==================== TEST QUERY ====================
print("\n" + "="*70)
print("PERFORMING TEST QUERY")
print("="*70)

if answers_collection.count() > 0 and questions_collection.count() > 0:
    # Get first question with its embedding
    first_question_data = questions_collection.get(
        limit=1,
        include=['embeddings', 'documents', 'metadatas']
    )
    
    # Check if we got data
    if first_question_data and first_question_data.get('documents') and len(first_question_data['documents']) > 0:
        test_question = first_question_data['documents'][0]
        test_embedding = first_question_data['embeddings'][0]
        
        print(f"\nTest Question: {test_question[:150]}...")
        print(f"Embedding shape: {len(test_embedding)} dimensions")
        
        # Query for similar answers
        print("\nSearching for similar answers...")
        results = answers_collection.query(
            query_embeddings=[test_embedding],
            n_results=5,
            include=['documents', 'metadatas', 'distances']
        )
        
        print(f"\nTop 5 most similar answers:\n")
        for i in range(len(results['documents'][0])):
            doc = results['documents'][0][i]
            metadata = results['metadatas'][0][i]
            distance = results['distances'][0][i]
            
            # Calculate similarity (cosine similarity from L2 distance)
            similarity = 1 - (distance ** 2) / 2
            
            print(f"{i+1}. Similarity: {similarity:.2%}")
            print(f"   Answer: {doc[:150]}...")
            print(f"   Row Index: {metadata.get('row_index', 'N/A')}")
            print(f"   Question: {metadata.get('question', 'N/A')[:100]}...")
            print()
        
        print("✓ Test query successful!")
        
        # ==================== TEST QUESTION-TO-QUESTION SIMILARITY ====================
        print("\n" + "="*70)
        print("TESTING QUESTION SIMILARITY")
        print("="*70)
        
        print(f"\nFinding questions similar to: {test_question[:100]}...")
        
        similar_questions = questions_collection.query(
            query_embeddings=[test_embedding],
            n_results=5,
            include=['documents', 'metadatas', 'distances']
        )
        
        print(f"\nTop 5 similar questions:\n")
        for i in range(len(similar_questions['documents'][0])):
            doc = similar_questions['documents'][0][i]
            metadata = similar_questions['metadatas'][0][i]
            distance = similar_questions['distances'][0][i]
            similarity = 1 - (distance ** 2) / 2
            
            print(f"{i+1}. Similarity: {similarity:.2%}")
            print(f"   Question: {doc[:150]}...")
            print(f"   Row Index: {metadata.get('row_index', 'N/A')}")
            print()
        
        print("✓ Question similarity test successful!")
        
    else:
        print("⚠ No data found in questions collection")
else:
    print("⚠ Collections are empty, cannot perform test")

# ==================== SAMPLE DOCUMENTS ====================
print("\n" + "="*70)
print("SAMPLE DOCUMENTS")
print("="*70)

print("\nSample Questions:")
sample_questions = questions_collection.get(
    limit=5,
    include=['documents', 'metadatas']
)

for i, (doc, meta) in enumerate(zip(sample_questions['documents'], sample_questions['metadatas'])):
    print(f"\n{i+1}. {doc[:100]}...")
    print(f"   Row: {meta.get('row_index', 'N/A')}")

print("\n\nSample Answers:")
sample_answers = answers_collection.get(
    limit=5,
    include=['documents', 'metadatas']
)

for i, (doc, meta) in enumerate(zip(sample_answers['documents'], sample_answers['metadatas'])):
    print(f"\n{i+1}. {doc[:100]}...")
    print(f"   Row: {meta.get('row_index', 'N/A')}")
    print(f"   Question: {meta.get('question', 'N/A')[:80]}...")

print("\n" + "="*70)
print("✓✓✓ TEST COMPLETE ✓✓✓")
print("="*70)

TESTING CHROMADB DATABASE

Connecting to ChromaDB...
✓ Connected to: vector_db

📊 Collection Status:
   Answers: 808 documents
   Questions: 808 documents

PERFORMING TEST QUERY

Test Question: heisenberg uncertainty principle...
Embedding shape: 768 dimensions

Searching for similar answers...

Top 5 most similar answers:

1. Similarity: 85.93%
   Answer: fermion type elementary particle electron quark make ordinary matter obey pauli exclusion principle state two fermion occupy quantum state time...
   Row Index: 633
   Question: fermion...

2. Similarity: 85.37%
   Answer: quantum mechanical principle state two identical fermion like electron occupy quantum state within system principle prevents atoms collapse give matte...
   Row Index: 635
   Question: pauli exclusion principle...

3. Similarity: 83.09%
   Answer: boson type elementary particle photon gluon act force carrier mediate fundamental force unlike fermion multiple boson occupy exact quantum state simul...
   Row Index: 63

In [3]:
import chromadb
import shutil
import os

vector_db_dir = "vector_db"

print("="*70)
print("CHROMADB DIAGNOSTIC & FIX")
print("="*70)

# Step 1: Check if database exists
if not os.path.exists(vector_db_dir):
    print(f"\n❌ Database directory does not exist: {vector_db_dir}")
    print("You need to run the embedding generation script first!")
    exit(1)

print(f"\n✓ Database directory exists: {vector_db_dir}")

# Step 2: Try to connect
print("\nAttempting to connect to ChromaDB...")
try:
    client = chromadb.PersistentClient(path=vector_db_dir)
    print("✓ Connected successfully")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\nTrying to fix...")
    
    # Remove lock files
    import glob
    lock_files = glob.glob(os.path.join(vector_db_dir, "**/*.lock"), recursive=True)
    for lock_file in lock_files:
        try:
            os.remove(lock_file)
            print(f"  Removed lock: {lock_file}")
        except:
            pass
    
    # Try again
    try:
        client = chromadb.PersistentClient(path=vector_db_dir)
        print("✓ Connected after cleanup")
    except Exception as e2:
        print(f"❌ Still failed: {e2}")
        print("\nRecommendation: Delete vector_db folder and re-run embedding script")
        exit(1)

# Step 3: Check collections
print("\n" + "="*70)
print("CHECKING COLLECTIONS")
print("="*70)

try:
    collections = client.list_collections()
    print(f"\nFound {len(collections)} collections:")
    for coll in collections:
        print(f"  - {coll.name}: {coll.count()} documents")
except Exception as e:
    print(f"❌ Error listing collections: {e}")
    exit(1)

# Step 4: Try to access specific collections
print("\n" + "="*70)
print("TESTING COLLECTION ACCESS")
print("="*70)

# Test all collections
all_collection_names = [coll.name for coll in collections]
required_collections = ["science_qa_answers", "science_qa_questions"]
collection_status = {}

print(f"\nAll collections: {all_collection_names}")
print(f"Required collections: {required_collections}")

for coll_name in all_collection_names:
    print(f"\n{'='*70}")
    print(f"Testing: {coll_name}")
    print(f"{'='*70}")
    try:
        coll = client.get_collection(coll_name)
        count = coll.count()
        print(f"  ✓ Collection exists: {count} documents")
        
        # Try to get a sample
        try:
            sample = coll.get(limit=1, include=['documents', 'embeddings', 'metadatas'])
            if sample['documents']:
                print(f"  ✓ Can read documents")
                print(f"    Sample doc: {sample['documents'][0][:100]}...")
                
                # FIXED: Check embeddings properly
                if sample['embeddings'] is not None and len(sample['embeddings']) > 0:
                    print(f"  ✓ Can read embeddings: dimension {len(sample['embeddings'][0])}")
                    
                    # Try a test query
                    print(f"  → Testing query...")
                    try:
                        test_results = coll.query(
                            query_embeddings=[sample['embeddings'][0]],
                            n_results=3,
                            include=['documents', 'distances']
                        )
                        print(f"  ✓ Query works! Got {len(test_results['documents'][0])} results")
                        print(f"    Top result distance: {test_results['distances'][0][0]:.4f}")
                        collection_status[coll_name] = "OK"
                    except Exception as qe:
                        print(f"  ❌ Query failed: {qe}")
                        collection_status[coll_name] = f"QUERY_ERROR: {str(qe)[:100]}"
                else:
                    print(f"  ❌ No embeddings found!")
                    collection_status[coll_name] = "NO_EMBEDDINGS"
                    
                if sample['metadatas']:
                    print(f"  ✓ Metadata: {sample['metadatas'][0]}")
            else:
                print(f"  ❌ Collection is empty!")
                collection_status[coll_name] = "EMPTY"
        except Exception as e:
            print(f"  ❌ Error reading data: {e}")
            import traceback
            print(f"  Traceback: {traceback.format_exc()}")
            collection_status[coll_name] = f"READ_ERROR: {str(e)[:100]}"
    except Exception as e:
        print(f"  ❌ Collection access failed: {e}")
        collection_status[coll_name] = f"ACCESS_ERROR: {str(e)[:100]}"

# Step 5: Summary and recommendations
print("\n" + "="*70)
print("SUMMARY & RECOMMENDATIONS")
print("="*70)

print("\n📊 Collection Status:")
for coll, status in collection_status.items():
    status_emoji = "✅" if status == "OK" else "❌"
    print(f"  {status_emoji} {coll}: {status}")

# Check which collections work
working_collections = [coll for coll, status in collection_status.items() if status == "OK"]
broken_collections = [coll for coll, status in collection_status.items() if status != "OK"]

if working_collections:
    print(f"\n✅ Working collections: {working_collections}")
    print(f"   You can use these collections in your application!")
    
    if "science_qa_answers" in working_collections and "science_qa_questions" in working_collections:
        print("\n✨ GREAT NEWS: Both required collections are working!")
    elif "qa_answers" in working_collections and "qa_questions" in working_collections:
        print("\n💡 TIP: The 'qa_answers' and 'qa_questions' collections are working.")
        print("   Consider using these instead of the science_qa_* versions.")

if broken_collections:
    print(f"\n❌ Broken collections: {broken_collections}")
    print("\n🔧 Recommended fixes:")
    
    for coll in broken_collections:
        status = collection_status[coll]
        print(f"\n  {coll}:")
        if "QUERY_ERROR" in status:
            if "Nothing found on disk" in status:
                print(f"    - Embeddings not properly saved to disk")
                print(f"    - Delete collection and regenerate:")
                print(f"      client.delete_collection('{coll}')")
                print(f"      Then re-run embedding script")
            else:
                print(f"    - Query operation failed")
                print(f"    - May need to regenerate embeddings")
        elif "NO_EMBEDDINGS" in status:
            print(f"    - Embeddings missing from collection")
            print(f"    - Re-run embedding generation script")
        elif "EMPTY" in status:
            print(f"    - Collection has no data")
            print(f"    - Re-run embedding script")
        else:
            print(f"    - Unknown issue: {status}")
            print(f"    - Try deleting and regenerating collection")

print("\n" + "="*70)

CHROMADB DIAGNOSTIC & FIX

✓ Database directory exists: vector_db

Attempting to connect to ChromaDB...
✓ Connected successfully

CHECKING COLLECTIONS

Found 4 collections:
  - science_qa_questions: 980 documents
  - qa_answers: 12354 documents
  - qa_questions: 12354 documents
  - science_qa_answers: 980 documents

TESTING COLLECTION ACCESS

All collections: ['science_qa_questions', 'qa_answers', 'qa_questions', 'science_qa_answers']
Required collections: ['science_qa_answers', 'science_qa_questions']

Testing: science_qa_questions
  ✓ Collection exists: 980 documents
  ✓ Can read documents
    Sample doc: heisenberg uncertainty principle...
  ✓ Can read embeddings: dimension 768
  → Testing query...
  ✓ Query works! Got 3 results
    Top result distance: 0.0000
  ✓ Metadata: {'row_index': 0, 'question_length': 3, 'answer': 'state position momentum particle precisely know simultaneously accurately one know less accurately'}

Testing: qa_answers
  ✓ Collection exists: 12354 documents
 

In [4]:
import chromadb
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ==================== CONFIGURATION ====================
vector_db_dir = "vector_db"
original_csv = "science_qa.csv"
preprocessed_csv = "preprocessed_science_qa.csv"

print("="*70)
print("CHROMADB Q&A MATCHING DIAGNOSTIC")
print("="*70)

# Load CSVs
df_original = pd.read_csv(original_csv)
df_preprocessed = pd.read_csv(preprocessed_csv)

print(f"\n✓ Loaded original CSV: {len(df_original)} rows")
print(f"✓ Loaded preprocessed CSV: {len(df_preprocessed)} rows")

# Find columns
def find_column(df, name):
    for col in df.columns:
        if col.lower() == name.lower():
            return col
    return None

question_col = find_column(df_original, 'question')
answer_col = find_column(df_original, 'answer')
question_col_prep = find_column(df_preprocessed, 'question')
answer_col_prep = find_column(df_preprocessed, 'answer')

# Connect to ChromaDB
client = chromadb.PersistentClient(path=vector_db_dir)
answers_collection = client.get_collection("science_qa_answers")
questions_collection = client.get_collection("science_qa_questions")

print(f"\n✓ Answers collection: {answers_collection.count()} documents")
print(f"✓ Questions collection: {questions_collection.count()} documents")

# ==================== TEST SPECIFIC EXAMPLES ====================
print("\n" + "="*70)
print("TESTING QUESTION-ANSWER MATCHING")
print("="*70)

# Test with first few rows
test_indices = [0, 1, 2, 5, 10]

for idx in test_indices:
    if idx >= len(df_original):
        continue
    
    print(f"\n{'='*70}")
    print(f"TEST ROW {idx}")
    print(f"{'='*70}")
    
    # Get original texts
    orig_question = df_original[question_col].iloc[idx]
    orig_answer = df_original[answer_col].iloc[idx]
    
    print(f"\n📝 Original Question:")
    print(f"   {orig_question[:100]}...")
    print(f"\n📝 Original Answer:")
    print(f"   {orig_answer[:100]}...")
    
    # Get preprocessed texts
    prep_question = df_preprocessed[question_col_prep].iloc[idx]
    prep_answer = df_preprocessed[answer_col_prep].iloc[idx]
    
    print(f"\n🔧 Preprocessed Question:")
    print(f"   {prep_question[:100]}...")
    print(f"\n🔧 Preprocessed Answer:")
    print(f"   {prep_answer[:100]}...")
    
    # Try to find question in DB
    try:
        # Get all questions and check if our row_idx exists
        all_questions = questions_collection.get(
            include=['documents', 'metadatas']
        )
        
        # Find matching metadata
        matching_q_idx = None
        for i, meta in enumerate(all_questions['metadatas']):
            if meta and 'row_index' in meta and meta['row_index'] == idx:
                matching_q_idx = i
                break
        
        if matching_q_idx is not None:
            db_question = all_questions['documents'][matching_q_idx]
            print(f"\n✓ Found question in DB (position {matching_q_idx}):")
            print(f"   {db_question[:100]}...")
            print(f"\n   Text match: {db_question == prep_question}")
        else:
            print(f"\n❌ Question with row_index={idx} NOT FOUND in DB!")
            continue
        
        # Get all answers and check if our row_idx exists
        all_answers = answers_collection.get(
            include=['documents', 'metadatas']
        )
        
        # Find matching metadata
        matching_a_idx = None
        for i, meta in enumerate(all_answers['metadatas']):
            if meta and 'row_index' in meta and meta['row_index'] == idx:
                matching_a_idx = i
                break
        
        if matching_a_idx is not None:
            db_answer = all_answers['documents'][matching_a_idx]
            print(f"\n✓ Found answer in DB (position {matching_a_idx}):")
            print(f"   {db_answer[:100]}...")
            print(f"\n   Text match: {db_answer == prep_answer}")
        else:
            print(f"\n❌ Answer with row_index={idx} NOT FOUND in DB!")
            continue
        
        # Now test the query mechanism
        print(f"\n{'─'*70}")
        print("TESTING QUERY MECHANISM")
        print(f"{'─'*70}")
        
        # Query with question embedding
        q_results = questions_collection.get(
            ids=[all_questions['ids'][matching_q_idx]],
            include=['embeddings']
        )
        
        if q_results['embeddings']:
            question_embedding = q_results['embeddings'][0]
            
            # Query answers collection
            a_results = answers_collection.query(
                query_embeddings=[question_embedding],
                n_results=3,
                include=['documents', 'metadatas', 'distances']
            )
            
            print(f"\nTop 3 answer matches when querying with question embedding:")
            for i, (doc, meta, dist) in enumerate(zip(
                a_results['documents'][0],
                a_results['metadatas'][0],
                a_results['distances'][0]
            )):
                similarity = 1 - (dist ** 2) / 2
                row_idx_result = meta.get('row_index', 'N/A') if meta else 'N/A'
                print(f"\n  Match {i+1}:")
                print(f"    Row index: {row_idx_result}")
                print(f"    Similarity: {similarity:.2%}")
                print(f"    Text: {doc[:80]}...")
                
                if i == 0:
                    if row_idx_result == idx:
                        print(f"    ✅ CORRECT! Matched to same row index")
                    else:
                        print(f"    ❌ WRONG! Expected row {idx}, got row {row_idx_result}")
                        print(f"    🚨 THIS IS THE BUG!")
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

# ==================== CHECK EMBEDDING CORRELATION ====================
print("\n" + "="*70)
print("CHECKING EMBEDDING STORAGE")
print("="*70)

# Get first 10 items from each collection
q_sample = questions_collection.get(
    limit=10,
    include=['documents', 'metadatas', 'ids']
)

a_sample = answers_collection.get(
    limit=10,
    include=['documents', 'metadatas', 'ids']
)

print("\nFirst 10 Questions in DB:")
for i, (doc, meta, id_) in enumerate(zip(q_sample['documents'], q_sample['metadatas'], q_sample['ids'])):
    row_idx = meta.get('row_index', 'N/A') if meta else 'N/A'
    print(f"  {i}: ID={id_}, row_index={row_idx}, text={doc[:50]}...")

print("\nFirst 10 Answers in DB:")
for i, (doc, meta, id_) in enumerate(zip(a_sample['documents'], a_sample['metadatas'], a_sample['ids'])):
    row_idx = meta.get('row_index', 'N/A') if meta else 'N/A'
    print(f"  {i}: ID={id_}, row_index={row_idx}, text={doc[:50]}...")

# ==================== SUMMARY ====================
print("\n" + "="*70)
print("DIAGNOSTIC SUMMARY")
print("="*70)

print("\n🔍 Check the following:")
print("  1. Do questions and answers have the SAME row_index in metadata?")
print("  2. When querying with a question embedding, does it return the")
print("     answer with the SAME row_index?")
print("  3. Are the IDs in sequential order or random?")
print("\n💡 If answers don't match questions by row_index, your embedding")
print("   generation script needs to be fixed!")
print("\n🔧 The fix: Ensure when generating embeddings, you store:")
print("   - Question embedding with metadata={'row_index': i}")
print("   - Answer embedding with metadata={'row_index': i}")
print("   Where i is the SAME index from the CSV")
print("="*70)

CHROMADB Q&A MATCHING DIAGNOSTIC

✓ Loaded original CSV: 980 rows
✓ Loaded preprocessed CSV: 980 rows

✓ Answers collection: 980 documents
✓ Questions collection: 980 documents

TESTING QUESTION-ANSWER MATCHING

TEST ROW 0

📝 Original Question:
   What is the Heisenberg Uncertainty Principle?...

📝 Original Answer:
   It states that the position and momentum of a particle cannot both be precisely known simultaneously...

🔧 Preprocessed Question:
   heisenberg uncertainty principle...

🔧 Preprocessed Answer:
   state position momentum particle precisely know simultaneously accurately one know less accurately...

✓ Found question in DB (position 0):
   heisenberg uncertainty principle...

   Text match: True

✓ Found answer in DB (position 0):
   state position momentum particle precisely know simultaneously accurately one know less accurately...

   Text match: True

──────────────────────────────────────────────────────────────────────
TESTING QUERY MECHANISM
──────────────────────────

Traceback (most recent call last):
  File "C:\Users\akqp4\AppData\Local\Temp\ipykernel_42228\900107719.py", line 131, in <module>
    if q_results['embeddings']:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Traceback (most recent call last):
  File "C:\Users\akqp4\AppData\Local\Temp\ipykernel_42228\900107719.py", line 131, in <module>
    if q_results['embeddings']:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Traceback (most recent call last):
  File "C:\Users\akqp4\AppData\Local\Temp\ipykernel_42228\900107719.py", line 131, in <module>
    if q_results['embeddings']:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Traceback (most recent call last):
  File "C:\Users\akqp4\AppData\Local\Temp\ipykernel_42228\900107719.py", line 131, in <module>
    if q_results['embeddings']:
ValueError: The truth value of an array 


✓ Found answer in DB (position 10):
   dna deoxyribonucleic acid molecule carry genetic instruction development function reproduction know ...

   Text match: True

──────────────────────────────────────────────────────────────────────
TESTING QUERY MECHANISM
──────────────────────────────────────────────────────────────────────

❌ Error: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

CHECKING EMBEDDING STORAGE


ValueError: Expected include item to be one of documents, embeddings, metadatas, distances, uris, data, got ids in get.

In [1]:
import re
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm


file_path = r"Preprocessed_QA_Dataset.csv"
output_dir = r"NLP"
model_name = "allenai/scibert_scivocab_uncased"
batch_size = 32 

# Device and model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

c:\Users\akqp4\python_projects\NLP\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [ ]:
#pair Question1 embedding with Answer1 embedding for similarity calculation
q1_embs = np.load("Question1_embeddings.npy")
a1_embs = np.load("Answer1_embeddings.npy")

# Cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
scores = cosine_similarity(q1_embs, a1_embs)  # row-wise similarity
